<a href="https://colab.research.google.com/github/wfvh/gan/blob/main/GAN_224pixels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pathlib
import numpy as np
import time
import random
import matplotlib.pyplot as plt
import PIL
import imageio
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import layers

def make_generator_model(noise_dim, img_width, img_height):
    model = tf.keras.Sequential()
    model.add(layers.InputLayer(input_shape=(noise_dim,)))

    model.add(layers.Dense(7*7*512, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Reshape((7, 7, 512)))
    assert model.output_shape == (None, 7, 7, 512)  # Note: None is the batch size


    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 512)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 28, 28, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 56, 56, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())


    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 112, 112, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, img_width, img_height, 3) #224,224,3

    return model
def make_discriminator_model(img_width, img_height):
    model = tf.keras.Sequential()

    model.add(layers.InputLayer(input_shape=(img_height, img_width, 3)))
    model.add(layers.GaussianDropout(0.05))
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1)) # no activation. logits=true when training
    return model

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = wasserstein_generator_loss(fake_output)
      disc_loss = wasserstein_discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for batch_of_images in dataset:
            train_step(batch_of_images)
        end = time.time()
        print ('Time for epoch {} is {} sec'.format(epoch + 1, round(end-start, 3)))

def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    time1 = time.time()
    predictions = model(test_input, training=False)
    fig123 = plt.figure()
    time2 = time.time()

    for i in range(predictions.shape[0]):
        ax1 = fig123.add_subplot(1, 1, i+1)
        image = tf.cast(tf.math.round(predictions[i, :, :, :] * 127.5 + 127.5), tf.int32)
        ax1.imshow(image)
        plt.axis('off')
    time3 = time.time()

    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    pil_image = PIL.Image.frombytes('RGB', canvas.get_width_height(), canvas.tostring_rgb())
    pil_image.save('image_at_epoch_{:04d}.png'.format(epoch))
    plt.close()
    #print("###Time for saving images {}, {}, {}".format(round(time2-time1,3), round(time3-time2,3), round(time.time()-time3,3)))
def generate_image():

    generated_image = tf.cast(tf.math.round(generator(tf.random.uniform([1, noise_dim]), training=False) * 127.5 + 127.5), tf.int32)[0]

    plt.imshow(generated_image)
    plt.axis('off')
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    pil_image = PIL.Image.frombytes('RGB', canvas.get_width_height(), canvas.tostring_rgb())
    pil_image.save("flower{}.png".format(round(time.time())))
    plt.close()
def generate_gif(frames=400, filename="evolution"):
    # Make a GIF of the current generator outputting images with a moving perlin noise
    anim_file = "{}{}.gif".format(filename, round(time.time()))
    perlin = (tf.random.uniform(shape=[1, noise_dim], minval=0.3, maxval=0.7))#(tf.ones([1, noise_dim]))
    with imageio.get_writer(anim_file, mode='I') as writer:
        for i in range(frames):
            # Make image
            array_image = np.array(tf.cast(tf.math.round(generator(perlin, training=False)[:, :, :] * 127.5 + 127.5), tf.uint8)[0, :, :, :])

            # Put image in writer for gif
            writer.append_data(array_image)
            # Change 1 bit, half at a time(so loop gif is 4times the noise worth of frames

            array = np.array(perlin)
            index = i % noise_dim
            #array[0][index] = (0.5 if int(array[0][index]) == 0.25 else (0.75 if int(array[0][index]) == 0.5 else (0.51 if int(array[0][index]) == 0.75 else 0.25)))
            randbit = (random.uniform(0.8,0.9) if random.choice([0,1])==1 else random.uniform(1.1,1.2))
            array[0][index] = array[0][index]*randbit
            perlin = tf.cast(array, tf.float32)
            if i % (frames / 5) == 0: print("{} % complete".format(i*100//frames))
        print("Finished. File called: {}".format(anim_file))

def getDataAndCache(dir='/birds', max=200, type='.jpg'):



    # Load Image Directory
    data_dir = pathlib.Path(str(dir))
    image_count = len(list(data_dir.glob('*/*'+type)))
    print("{} images in the set.".format(image_count))
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        seed=123,
        image_size=(img_height, img_width),
        labels=None,
        batch_size=min(max,image_count-image_count%BATCH_SIZE))

    image_batch = next(iter(train_ds))

    # Prepare data
    train_images = (image_batch - 127.5) / 127.5  # Normalize the images to [-1, 1]

    train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    print("With {} batches of {} images each.".format(len(train_dataset), BATCH_SIZE))
    # .cache keeps images in memory during training.
    train_dataset = train_dataset.cache()
    return train_dataset

def production(preschool = 2, rate = 2, produce=1, time_allowed=60, gif=True, image=True):
    start = time.time()

# early training period to get it up to scratch
    print("Starting preschool for {} epochs".format(preschool))
    train(train_dataset, preschool)


# Production period, pump out the gifs and photos
    for i in range(produce):
        if time.time()-start<time_allowed:
            print("Production lot {} starting. Training for {} epochs.".format(i+1,rate))
            train(train_dataset, rate)

            if gif: generate_gif()
            if image: generate_image()

            print("Production lot {} finished.".format(i+1))
        else:
            print("Finished due to time constraints.")
            break

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf

__all__ = [
    'acgan_discriminator_loss',
    'acgan_generator_loss',
    'least_squares_discriminator_loss',
    'least_squares_generator_loss',
    'modified_discriminator_loss',
    'modified_generator_loss',
    'minimax_discriminator_loss',
    'minimax_generator_loss',
    'wasserstein_discriminator_loss',
    'wasserstein_hinge_generator_loss',
    'wasserstein_hinge_discriminator_loss',
    'wasserstein_generator_loss',
    'wasserstein_gradient_penalty',
    'mutual_information_penalty',
    'combine_adversarial_loss',
    'cycle_consistency_loss',
]


def _to_float(tensor):
  return tf.cast(tensor, tf.float32)


# Wasserstein losses from `Wasserstein GAN` (https://arxiv.org/abs/1701.07875).
def wasserstein_generator_loss(
    discriminator_gen_outputs,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  with tf.compat.v1.name_scope(scope, 'generator_wasserstein_loss',
                               (discriminator_gen_outputs, weights)) as scope:
    discriminator_gen_outputs = _to_float(discriminator_gen_outputs)

    loss = - discriminator_gen_outputs
    loss = tf.compat.v1.losses.compute_weighted_loss(loss, weights, scope,
                                                     loss_collection, reduction)

    if add_summaries:
      tf.compat.v1.summary.scalar('generator_wass_loss', loss)

  return loss


def wasserstein_discriminator_loss(
    discriminator_real_outputs,
    discriminator_gen_outputs,
    real_weights=1.0,
    generated_weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):
  with tf.compat.v1.name_scope(
      scope, 'discriminator_wasserstein_loss',
      (discriminator_real_outputs, discriminator_gen_outputs, real_weights,
       generated_weights)) as scope:
    discriminator_real_outputs = _to_float(discriminator_real_outputs)
    discriminator_gen_outputs = _to_float(discriminator_gen_outputs)
    discriminator_real_outputs.shape.assert_is_compatible_with(
        discriminator_gen_outputs.shape)

    loss_on_generated = tf.compat.v1.losses.compute_weighted_loss(
        discriminator_gen_outputs,
        generated_weights,
        scope,
        loss_collection=None,
        reduction=reduction)
    loss_on_real = tf.compat.v1.losses.compute_weighted_loss(
        discriminator_real_outputs,
        real_weights,
        scope,
        loss_collection=None,
        reduction=reduction)
    loss = loss_on_generated - loss_on_real
    tf.compat.v1.losses.add_loss(loss, loss_collection)

    if add_summaries:
      tf.compat.v1.summary.scalar('discriminator_gen_wass_loss',
                                  loss_on_generated)
      tf.compat.v1.summary.scalar('discriminator_real_wass_loss', loss_on_real)
      tf.compat.v1.summary.scalar('discriminator_wass_loss', loss)

  return loss


wasserstein_hinge_generator_loss = wasserstein_generator_loss
wasserstein_hinge_generator_loss.__name__ = 'wasserstein_hinge_generator_loss'


def wasserstein_hinge_discriminator_loss(
    discriminator_real_outputs,
    discriminator_gen_outputs,
    real_weights=1.0,
    generated_weights=1.0,
    real_hinge=1.0,
    generated_hinge=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  with tf.compat.v1.name_scope(
      scope, 'discriminator_wasserstein_hinge_loss',
      (discriminator_real_outputs, discriminator_gen_outputs, real_weights,
       generated_weights)) as scope:
    discriminator_real_outputs = _to_float(discriminator_real_outputs)
    discriminator_gen_outputs = _to_float(discriminator_gen_outputs)
    discriminator_real_outputs.shape.assert_is_compatible_with(
        discriminator_gen_outputs.shape)

    # Compute the hinge.
    hinged_real = tf.nn.relu(real_hinge - discriminator_real_outputs)
    hinged_gen = tf.nn.relu(generated_hinge + discriminator_gen_outputs)

    # Average.
    loss_on_real = tf.compat.v1.losses.compute_weighted_loss(
        hinged_real,
        real_weights,
        scope,
        loss_collection=None,
        reduction=reduction)
    loss_on_generated = tf.compat.v1.losses.compute_weighted_loss(
        hinged_gen,
        generated_weights,
        scope,
        loss_collection=None,
        reduction=reduction)
    loss = loss_on_generated + loss_on_real
    tf.compat.v1.losses.add_loss(loss, loss_collection)

    if add_summaries:
      tf.compat.v1.summary.scalar('discriminator_gen_wass_hinge_loss',
                                  loss_on_generated)
      tf.compat.v1.summary.scalar('discriminator_real_wass_hinge_loss',
                                  loss_on_real)
      tf.compat.v1.summary.scalar('discriminator_wass_hinge_loss', loss)

  return loss


# ACGAN losses from `Conditional Image Synthesis With Auxiliary Classifier GANs`
# (https://arxiv.org/abs/1610.09585).
def acgan_discriminator_loss(
    discriminator_real_classification_logits,
    discriminator_gen_classification_logits,
    one_hot_labels,
    label_smoothing=0.0,
    real_weights=1.0,
    generated_weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):
  
  with tf.compat.v1.name_scope(
      scope, 'acgan_discriminator_loss',
      (discriminator_real_classification_logits,
       discriminator_gen_classification_logits, one_hot_labels)) as scope:
    loss_on_generated = tf.compat.v1.losses.softmax_cross_entropy(
        one_hot_labels,
        discriminator_gen_classification_logits,
        weights=generated_weights,
        scope=scope,
        loss_collection=None,
        reduction=reduction)
    loss_on_real = tf.compat.v1.losses.softmax_cross_entropy(
        one_hot_labels,
        discriminator_real_classification_logits,
        weights=real_weights,
        label_smoothing=label_smoothing,
        scope=scope,
        loss_collection=None,
        reduction=reduction)
    loss = loss_on_generated + loss_on_real
    tf.compat.v1.losses.add_loss(loss, loss_collection)

    if add_summaries:
      tf.compat.v1.summary.scalar('discriminator_gen_ac_loss',
                                  loss_on_generated)
      tf.compat.v1.summary.scalar('discriminator_real_ac_loss', loss_on_real)
      tf.compat.v1.summary.scalar('discriminator_ac_loss', loss)

  return loss


def acgan_generator_loss(
    discriminator_gen_classification_logits,
    one_hot_labels,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):
  
  with tf.compat.v1.name_scope(
      scope, 'acgan_generator_loss',
      (discriminator_gen_classification_logits, one_hot_labels)) as scope:
    loss = tf.compat.v1.losses.softmax_cross_entropy(
        one_hot_labels,
        discriminator_gen_classification_logits,
        weights=weights,
        scope=scope,
        loss_collection=loss_collection,
        reduction=reduction)

    if add_summaries:
      tf.compat.v1.summary.scalar('generator_ac_loss', loss)

  return loss



def wasserstein_gradient_penalty(
    real_data,
    generated_data,
    generator_inputs,
    discriminator_fn,
    discriminator_scope,
    epsilon=1e-10,
    target=1.0,
    one_sided=False,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  if tf.executing_eagerly():
    raise RuntimeError('Can\'t use `tf.gradient` when executing eagerly.')
  with tf.compat.v1.name_scope(scope, 'wasserstein_gradient_penalty',
                               (real_data, generated_data)) as scope:
    real_data = tf.convert_to_tensor(value=real_data)
    generated_data = tf.convert_to_tensor(value=generated_data)
    if real_data.shape.ndims is None:
      raise ValueError('`real_data` can\'t have unknown rank.')
    if generated_data.shape.ndims is None:
      raise ValueError('`generated_data` can\'t have unknown rank.')

    differences = generated_data - real_data
    batch_size = (tf.compat.dimension_value(differences.shape.dims[0]) or
                  tf.shape(input=differences)[0])
    alpha_shape = [batch_size] + [1] * (differences.shape.ndims - 1)
    alpha = tf.random.uniform(shape=alpha_shape)
    interpolates = real_data + (alpha * differences)

    with tf.compat.v1.name_scope(
        ''):  # Clear scope so update ops are added properly.
      # Reuse variables if variables already exists.
      with tf.compat.v1.variable_scope(
          discriminator_scope, 'gpenalty_dscope',
          reuse=tf.compat.v1.AUTO_REUSE):
        disc_interpolates = discriminator_fn(interpolates, generator_inputs)

    if isinstance(disc_interpolates, tuple):
      # ACGAN case: disc outputs more than one tensor
      disc_interpolates = disc_interpolates[0]

    gradients = tf.gradients(ys=disc_interpolates, xs=interpolates)[0]
    gradient_squares = tf.reduce_sum(
        input_tensor=tf.square(gradients),
        axis=list(range(1, gradients.shape.ndims)))
    # Propagate shape information, if possible.
    if isinstance(batch_size, int):
      gradient_squares.set_shape([
          batch_size] + gradient_squares.shape.as_list()[1:])
    # For numerical stability, add epsilon to the sum before taking the square
    # root. Note tf.norm does not add epsilon.
    slopes = tf.sqrt(gradient_squares + epsilon)
    penalties = slopes / target - 1.0
    if one_sided:
      penalties = tf.maximum(0., penalties)
    penalties_squared = tf.square(penalties)
    penalty = tf.compat.v1.losses.compute_weighted_loss(
        penalties_squared,
        weights,
        scope=scope,
        loss_collection=loss_collection,
        reduction=reduction)

    if add_summaries:
      tf.compat.v1.summary.scalar('gradient_penalty_loss', penalty)

    return penalty




def minimax_discriminator_loss(
    discriminator_real_outputs,
    discriminator_gen_outputs,
    label_smoothing=0.25,
    real_weights=1.0,
    generated_weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  with tf.compat.v1.name_scope(
      scope, 'discriminator_minimax_loss',
      (discriminator_real_outputs, discriminator_gen_outputs, real_weights,
       generated_weights, label_smoothing)) as scope:

    # -log((1 - label_smoothing) - sigmoid(D(x)))
    loss_on_real = tf.compat.v1.losses.sigmoid_cross_entropy(
        tf.ones_like(discriminator_real_outputs),
        discriminator_real_outputs,
        real_weights,
        label_smoothing,
        scope,
        loss_collection=None,
        reduction=reduction)
    # -log(- sigmoid(D(G(x))))
    loss_on_generated = tf.compat.v1.losses.sigmoid_cross_entropy(
        tf.zeros_like(discriminator_gen_outputs),
        discriminator_gen_outputs,
        generated_weights,
        scope=scope,
        loss_collection=None,
        reduction=reduction)

    loss = loss_on_real + loss_on_generated
    tf.compat.v1.losses.add_loss(loss, loss_collection)

    if add_summaries:
      tf.compat.v1.summary.scalar('discriminator_gen_minimax_loss',
                                  loss_on_generated)
      tf.compat.v1.summary.scalar('discriminator_real_minimax_loss',
                                  loss_on_real)
      tf.compat.v1.summary.scalar('discriminator_minimax_loss', loss)

  return loss


def minimax_generator_loss(
    discriminator_gen_outputs,
    label_smoothing=0.0,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  with tf.compat.v1.name_scope(scope, 'generator_minimax_loss') as scope:
    loss = - minimax_discriminator_loss(
        tf.ones_like(discriminator_gen_outputs),
        discriminator_gen_outputs, label_smoothing, weights, weights, scope,
        loss_collection, reduction, add_summaries=False)

  if add_summaries:
    tf.compat.v1.summary.scalar('generator_minimax_loss', loss)

  return loss


def modified_discriminator_loss(
    discriminator_real_outputs,
    discriminator_gen_outputs,
    label_smoothing=0.25,
    real_weights=1.0,
    generated_weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  return minimax_discriminator_loss(
      discriminator_real_outputs,
      discriminator_gen_outputs,
      label_smoothing,
      real_weights,
      generated_weights,
      scope or 'discriminator_modified_loss',
      loss_collection,
      reduction,
      add_summaries)


def modified_generator_loss(
    discriminator_gen_outputs,
    label_smoothing=0.0,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):


  with tf.compat.v1.name_scope(scope, 'generator_modified_loss',
                               [discriminator_gen_outputs]) as scope:
    loss = tf.compat.v1.losses.sigmoid_cross_entropy(
        tf.ones_like(discriminator_gen_outputs), discriminator_gen_outputs,
        weights, label_smoothing, scope, loss_collection, reduction)

    if add_summaries:
      tf.compat.v1.summary.scalar('generator_modified_loss', loss)

  return loss





def least_squares_generator_loss(
    discriminator_gen_outputs,
    real_label=1,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  with tf.compat.v1.name_scope(
      scope, 'lsq_generator_loss',
      (discriminator_gen_outputs, real_label)) as scope:
    discriminator_gen_outputs = _to_float(discriminator_gen_outputs)
    loss = tf.math.squared_difference(discriminator_gen_outputs,
                                      real_label) / 2.0
    loss = tf.compat.v1.losses.compute_weighted_loss(loss, weights, scope,
                                                     loss_collection, reduction)

  if add_summaries:
    tf.compat.v1.summary.scalar('generator_lsq_loss', loss)

  return loss


def least_squares_discriminator_loss(
    discriminator_real_outputs,
    discriminator_gen_outputs,
    real_label=1,
    fake_label=0,
    real_weights=1.0,
    generated_weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):

  with tf.compat.v1.name_scope(
      scope, 'lsq_discriminator_loss',
      (discriminator_gen_outputs, real_label)) as scope:
    discriminator_real_outputs = _to_float(discriminator_real_outputs)
    discriminator_gen_outputs = _to_float(discriminator_gen_outputs)
    discriminator_real_outputs.shape.assert_is_compatible_with(
        discriminator_gen_outputs.shape)

    real_losses = tf.math.squared_difference(discriminator_real_outputs,
                                             real_label) / 2.0
    fake_losses = tf.math.squared_difference(discriminator_gen_outputs,
                                             fake_label) / 2.0

    loss_on_real = tf.compat.v1.losses.compute_weighted_loss(
        real_losses,
        real_weights,
        scope,
        loss_collection=None,
        reduction=reduction)
    loss_on_generated = tf.compat.v1.losses.compute_weighted_loss(
        fake_losses,
        generated_weights,
        scope,
        loss_collection=None,
        reduction=reduction)

    loss = loss_on_real + loss_on_generated
    tf.compat.v1.losses.add_loss(loss, loss_collection)

  if add_summaries:
    tf.compat.v1.summary.scalar('discriminator_gen_lsq_loss', loss_on_generated)
    tf.compat.v1.summary.scalar('discriminator_real_lsq_loss', loss_on_real)
    tf.compat.v1.summary.scalar('discriminator_lsq_loss', loss)

  return loss




def _validate_distributions(distributions):
  """Check that input is a distribution."""
  if not isinstance(distributions, (list, tuple)):
    raise ValueError('`distributions` must be a list or tuple. Instead, '
                     'found %s.' % type(distributions))
  for x in distributions:
    if not callable(getattr(x, 'log_prob', None)):
      raise ValueError('`distributions` must be a list of `Distributions`. '
                       'Instead, found %s.' % type(x))


def _validate_information_penalty_inputs(
    structured_generator_inputs, predicted_distributions):
  """Validate input to `mutual_information_penalty`."""
  _validate_distributions(predicted_distributions)
  if len(structured_generator_inputs) != len(predicted_distributions):
    raise ValueError('`structured_generator_inputs` length %i must be the same '
                     'as `predicted_distributions` length %i.' % (
                         len(structured_generator_inputs),
                         len(predicted_distributions)))


def mutual_information_penalty(
    structured_generator_inputs,
    predicted_distributions,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):
  
  _validate_information_penalty_inputs(
      structured_generator_inputs, predicted_distributions)

  with tf.compat.v1.name_scope(scope, 'mutual_information_loss') as scope:
    # Calculate the negative log-likelihood of the reconstructed noise.
    log_probs = [
        tf.reduce_mean(input_tensor=dist.log_prob(noise)) for dist, noise in
        zip(predicted_distributions, structured_generator_inputs)
    ]
    loss = -1 * tf.compat.v1.losses.compute_weighted_loss(
        log_probs,
        weights,
        scope,
        loss_collection=loss_collection,
        reduction=reduction)

    if add_summaries:
      tf.compat.v1.summary.scalar('mutual_information_penalty', loss)

  return loss


def numerically_stable_global_norm(tensor_list):
  if all(x is None for x in tensor_list):
    return 0.0

  list_max = tf.reduce_max(input_tensor=[
      tf.reduce_max(input_tensor=tf.abs(x))
      for x in tensor_list
      if x is not None
  ])
  return list_max * tf.linalg.global_norm(
      [x / list_max for x in tensor_list if x is not None])


def _used_weight(weights_list):
  for weight in weights_list:
    if weight is not None:
      return tf.get_static_value(tf.convert_to_tensor(value=weight))


def _validate_args(weight_factor, gradient_ratio):
  if weight_factor is None and gradient_ratio is None:
    raise ValueError(
        '`weight_factor` and `gradient_ratio` cannot both be `None.`')
  if weight_factor is not None and gradient_ratio is not None:
    raise ValueError(
        '`weight_factor` and `gradient_ratio` cannot both be specified.')


# TODO(joelshor): Add ability to pass in gradients, to avoid recomputing.
def combine_adversarial_loss(main_loss,
                             adversarial_loss,
                             weight_factor=None,
                             gradient_ratio=None,
                             gradient_ratio_epsilon=1e-6,
                             variables=None,
                             scalar_summaries=True,
                             gradient_summaries=True,
                             scope=None):
 
  _validate_args(weight_factor, gradient_ratio)
  if variables is None:
    variables = contrib.get_trainable_variables()

  with tf.compat.v1.name_scope(
      scope, 'adversarial_loss', values=[main_loss, adversarial_loss]):
    # If losses are not the same shape, reduce them to both be shape [batch,].
    if not main_loss.shape.is_compatible_with(adversarial_loss.shape):
      if main_loss.shape[0] != adversarial_loss.shape[0]:
        raise ValueError(
            'main_loss and adversarial_loss must have the same sized first '
            'dimension. Found %d and %d.' %
            (main_loss.shape[0], adversarial_loss.shape[0]))
      tf.compat.v1.logging.warning(
          'Applying mean reduction per-batch-element to main and adversarial '
          'losses to make shapes compatible. If this is undesirable, ensure '
          'that the shapes are compatible before passing them into '
          'combine_adversarial_loss.')
      main_loss = tf.math.reduce_mean(
          input_tensor=main_loss, axis=list(range(1, main_loss.shape.rank)))
      adversarial_loss = tf.math.reduce_mean(
          input_tensor=adversarial_loss,
          axis=list(range(1, adversarial_loss.shape.rank)))

    # Compute gradients if we will need them.
    if gradient_summaries or gradient_ratio is not None:
      # `tf.gradients` doesn't work in eager.
      if tf.executing_eagerly():
        raise RuntimeError('`tf.gradients` doesn\'t work in eager.')
      main_loss_grad_mag = numerically_stable_global_norm(
          tf.gradients(ys=main_loss, xs=variables))
      adv_loss_grad_mag = numerically_stable_global_norm(
          tf.gradients(ys=adversarial_loss, xs=variables))

    # Add summaries, if applicable.
    if scalar_summaries:
      tf.compat.v1.summary.scalar('main_loss',
                                  tf.math.reduce_mean(input_tensor=main_loss))
      tf.compat.v1.summary.scalar(
          'adversarial_loss',
          tf.math.reduce_mean(input_tensor=adversarial_loss))
    if gradient_summaries:
      tf.compat.v1.summary.scalar('main_loss_gradients', main_loss_grad_mag)
      tf.compat.v1.summary.scalar('adversarial_loss_gradients',
                                  adv_loss_grad_mag)

    # Combine losses in the appropriate way.
    # If `weight_factor` is always `0`, avoid computing the adversarial loss
    # tensor entirely.
    if _used_weight((weight_factor, gradient_ratio)) == 0:
      final_loss = main_loss
    elif weight_factor is not None:
      final_loss = (main_loss +
                    tf.stop_gradient(weight_factor) * adversarial_loss)
    elif gradient_ratio is not None:
      grad_mag_ratio = main_loss_grad_mag / (
          adv_loss_grad_mag + gradient_ratio_epsilon)
      adv_coeff = grad_mag_ratio / gradient_ratio
      tf.compat.v1.summary.scalar('adversarial_coefficient', adv_coeff)
      final_loss = (main_loss +
                    tf.stop_gradient(adv_coeff) * adversarial_loss)

  return final_loss


def cycle_consistency_loss(data_x,
                           reconstructed_data_x,
                           data_y,
                           reconstructed_data_y,
                           scope=None,
                           add_summaries=False):
  

  with tf.compat.v1.name_scope(
      scope,
      'cycle_consistency_loss',
      values=[data_x, reconstructed_data_x, data_y, reconstructed_data_y]):
    loss_x2x = tf.compat.v1.losses.absolute_difference(data_x,
                                                       reconstructed_data_x)
    loss_y2y = tf.compat.v1.losses.absolute_difference(data_y,
                                                       reconstructed_data_y)
    loss = (loss_x2x + loss_y2y) / 2.0
    if add_summaries:
      tf.compat.v1.summary.scalar('cycle_consistency_loss_x2x', loss_x2x)
      tf.compat.v1.summary.scalar('cycle_consistency_loss_y2y', loss_y2y)
      tf.compat.v1.summary.scalar('cycle_consistency_loss', loss)

  return loss





In [29]:
# Define Variables
img_height = 224
img_width = 224
noise_dim = 100
BUFFER_SIZE = 60000
BATCH_SIZE=10


In [30]:
# Check GPU
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7859664435960541848
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11345264640
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7202496017301362447
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [32]:
# Create and Cache training dataset
train_dataset = getDataAndCache(dir='birds')

119 images in the set.
Found 119 files belonging to 1 classes.
With 11 batches of 10 images each.


In [ ]:

# Create the Models
generator = make_generator_model(noise_dim, img_width, img_height)
discriminator = make_discriminator_model(img_width, img_height)

# Create learning Optimisers for each model
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5)

# Print summary for each model so we can see the layers n whatnot.
generator.summary()
discriminator.summary()

In [ ]:
# Start the production process. Involves a preschool training phase where nothing is generated.
# Then it produces 'produce' amount of sets of images, every 'rate' epochs.
# Limit time allowed with the time_allowed variable.
production(preschool = 100, rate = 100, produce=20, time_allowed=1*60*60, gif=False)


Starting preschool for 100 epochs
Time for epoch 1 is 3.942 sec
Time for epoch 2 is 4.053 sec
Time for epoch 3 is 4.134 sec
Time for epoch 4 is 4.136 sec
Time for epoch 5 is 4.115 sec
Time for epoch 6 is 4.13 sec
Time for epoch 7 is 4.114 sec
Time for epoch 8 is 4.122 sec
Time for epoch 9 is 4.109 sec
Time for epoch 10 is 4.117 sec
Time for epoch 11 is 4.102 sec
Time for epoch 12 is 4.105 sec
Time for epoch 13 is 4.112 sec
Time for epoch 14 is 4.096 sec
Time for epoch 15 is 4.097 sec
Time for epoch 16 is 4.1 sec
Time for epoch 17 is 4.103 sec
Time for epoch 18 is 4.101 sec
Time for epoch 19 is 4.093 sec
Time for epoch 20 is 4.093 sec
Time for epoch 21 is 5.156 sec
Time for epoch 22 is 3.765 sec
Time for epoch 23 is 4.095 sec
Time for epoch 24 is 4.084 sec
Time for epoch 25 is 4.105 sec
Time for epoch 26 is 4.1 sec
Time for epoch 27 is 4.102 sec
Time for epoch 28 is 4.094 sec
Time for epoch 29 is 4.107 sec
Time for epoch 30 is 4.1 sec
Time for epoch 31 is 4.1 sec
Time for epoch 32 is 4.